# Load conf files

In [11]:
import tomli
import re
from pydantic import BaseModel
from uuid import uuid4
inputs_conf = 'config/inputs.conf'

schema = BaseModel
with open(file) as f:
    tmp = f.read()
    # .conf files aren't exactly TOML, so we need to format it a bit
    # first before we can load it
    tmp_formatted = re.sub(r"(\s*=\s*)(.*)", r"\1'\2'", tmp)
    tomli_inputs = tomli.loads(tmp_formatted).items()


In [12]:
tomli_inputs

dict_items([('assignmentgroups', {'connection': 'TOOLX_P', 'disabled': '0', 'host': 'host1', 'index': 'indexx', 'index_time_mode': 'current', 'interval': '0 6 * * *', 'max_rows': '100000', 'mode': 'batch', 'query': 'SELECT * FROM "OWNER"."ASSIGNMENT_GROUPS_SPLUNK"', 'source': 'toolx', 'sourcetype': 'assignmentgroups'}), ('CI_relations', {'connection': 'TOOLX_P', 'disabled': '0', 'host': 'host1', 'index': 'indexx', 'index_time_mode': 'current', 'interval': '0 5 * * *', 'max_rows': '1000000', 'mode': 'batch', 'query': 'SELECT * FROM "OWNER"."RELATIONSHIP_SPLUNK"', 'source': 'toolx', 'sourcetype': 'ci_relations'}), ('config_full', {'connection': 'TOOLY_P', 'description': 'Get all config daily', 'disabled': '0', 'host': 'host2', 'index': 'indexy', 'index_time_mode': 'current', 'interval': '0 4 * * *', 'max_rows': '1000000', 'mode': 'batch', 'query': 'SELECT * FROM "OWNER"."CONFIG_SPLUNK"', 'query_timeout': '300', 'source': 'tooly', 'sourcetype': 'tooly'})])

# Re organize data to make 2 batches, one for the Input API and another for Connections API

# Schemas that include aliases

In [ ]:
class ConnectionSchema(Basemodel):
    id: str
    databaseType: str
    username: str
    password: str
    connectionString: str | None = None
    database: str
    disabled: int
    host: str
    identity: str
    jdbcUseSSL: bool
    localTimezoneConversionEnabled: bool
    port: int
    readonly: bool
    timezone: str | None = None
    

In [ ]:
class Metadata(BaseModel):
    name: str
    value: str

class Schedule(BaseModel):
    cronSchedule: str 
    run: Dict[str, str]
    enabled: bool

class CollectorConf(BaseModel):
    connectionId: str
    query: str

class Collector(BaseModel):
    conf: CollectorConf
    type: str

class InputType(BaseModel):
    type: str
    metadata: List[Metadata]

class InputSchema(BaseModel):
    type: str | None = None
    schedule: Schedule
    collector: Collector
    input: InputType
    id: str| None = None

In [ ]:
# Two load functions, one for the Iputs and one for Connections